# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, f1_score

In [2]:
data = pd.read_csv('data/labeled.csv')

In [3]:
class Experiment:

    train, test = train_test_split(data, test_size=0.1, shuffle=True)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)

    def __init__(
            self,
            vectorizer_model: object,
            vectorizer_args : dict,
            classifier_model: object,
            classifier_args : dict
        ):
        
        self.vectorizer = vectorizer_model(**vectorizer_args)
        X = self.vectorizer.fit_transform(self.train.comment)
        y = self.train.toxic.values

        self.classifier = classifier_model(**classifier_args)
        self.classifier.fit(X, y)

    def get_test_preds(self):
        X = self.vectorizer.transform(self.test.comment)
        preds = self.classifier.predict(X)
        return preds

    def get_report(self):
        return classification_report(
            self.test.toxic.values,
            self.get_test_preds(),
            zero_division=0
        )
    
    def get_f1(self):
        return f1_score(
            self.test.toxic.values,
            self.get_test_preds(),
            zero_division=0
        )

In [4]:
experiment_default = Experiment(
    TfidfVectorizer,
    {},
    MultinomialNB,
    {}
)

In [5]:
print(
    experiment_default.get_report()
)

              precision    recall  f1-score   support

         0.0       0.73      1.00      0.84       959
         1.0       0.97      0.26      0.41       483

    accuracy                           0.75      1442
   macro avg       0.85      0.63      0.62      1442
weighted avg       0.81      0.75      0.69      1442



In [6]:
import razdel

In [7]:
## Если просто сунуть по ссылке, прилетает TypeError: '<' not supported between instances of 'Substring' and 'Substring'

def odel_tokenize(text):
    return [_.text for _ in razdel.tokenize(text)]

In [8]:
experiment_razdel = Experiment(
    TfidfVectorizer,
    {
        "tokenizer": odel_tokenize,
        "analyzer" : "word"
    },
    MultinomialNB,
    {}
)

/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
print(
    experiment_razdel.get_report()
)

              precision    recall  f1-score   support

         0.0       0.71      1.00      0.83       959
         1.0       0.98      0.20      0.33       483

    accuracy                           0.73      1442
   macro avg       0.85      0.60      0.58      1442
weighted avg       0.80      0.73      0.66      1442



#### Комментарий

В документации Razdel написано, что

<q>
Правила в Razdel оптимизированы для аккуратно написанных текстов с правильной пунктуацией. Решение хорошо работает с новостными статьями, художественными текстами. На постах из социальных сетей, расшифровках телефонных разговоров качество ниже.
</q>

Поэтому неудивительно, что на данном датасете Razdel работает чуть хуже.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [11]:
experiment_params = (

    {
        "classifier_model": MultinomialNB,
        "classifier_args" : {},  ##  Чудесным образом любое вмешательство в парамеры сильно ухудшает результаты
        "vectorizer_model": CountVectorizer,
        ##  Здесь и далее наилучшие результаты давала векторизация с параметрами analyzer + ngram_range + max_df
        ##  Т.е. с 3-мя вручную заданными параметрами, поэтому 2 отставшихся по заданию забил дефолтными.
        ##  Иные вмешательства делали результат хуже.
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    ##  Фактически задание не выполнено т.к. в обоих случаях результаты получаются лучше с мешком слов.
    ##  Однако вот такая конфигурация проходит только с варнингом, поэтому можно считать, что TF-IDF лучше т.к. стабильнее (?)
    {   
        "classifier_model": LogisticRegression,
        "classifier_args" : {"solver":"liblinear", "penalty":"l2"},
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {
        "classifier_model": LogisticRegression,
        "classifier_args" : {"solver":"liblinear", "penalty":"l2"},
        "vectorizer_model": TfidfVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

)

In [12]:
finished_experiments = []

for d in experiment_params:
    experiment = Experiment(**d)
    print(f'{d["classifier_model"].__name__}\t{d["vectorizer_model"].__name__}')
    print("F1: ", experiment.get_f1())
    print(experiment.get_report())
    finished_experiments.append(experiment)

MultinomialNB	CountVectorizer
F1:  0.8726114649681529
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94       959
         1.0       0.90      0.85      0.87       483

    accuracy                           0.92      1442
   macro avg       0.91      0.90      0.91      1442
weighted avg       0.92      0.92      0.92      1442



/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LogisticRegression	CountVectorizer
F1:  0.837108953613808
              precision    recall  f1-score   support

         0.0       0.90      0.94      0.92       959
         1.0       0.87      0.80      0.84       483

    accuracy                           0.90      1442
   macro avg       0.89      0.87      0.88      1442
weighted avg       0.89      0.90      0.89      1442

LogisticRegression	TfidfVectorizer
F1:  0.8216340621403913
              precision    recall  f1-score   support

         0.0       0.88      0.97      0.92       959
         1.0       0.92      0.74      0.82       483

    accuracy                           0.89      1442
   macro avg       0.90      0.85      0.87      1442
weighted avg       0.90      0.89      0.89      1442



In [13]:
##  Для читаемости
nb_bow_preds       = finished_experiments[0].get_test_preds()
logreg_bow_preds   = finished_experiments[1].get_test_preds()
logreg_tfidf_preds = finished_experiments[2].get_test_preds()

In [100]:
from collections import Counter

In [119]:
def get_10_most_toxic(texts:list[str], experiment:Experiment):
    vectors = experiment.vectorizer.transform(texts)
    probas = experiment.classifier.predict_proba(vectors)
    counter = Counter({
        text: proba[1]
        for text, proba
        in zip(
            texts, probas
        )
    })

    return counter.most_common(10)

In [120]:
get_10_most_toxic(data.comment, finished_experiments[0])

[('Верблюдов-то за что? Дебилы, бл...\n', np.float64(1.0)),
 ('Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n',
  np.float64(1.0)),
 ('Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n',
  np.float64(1.0)),
 ('тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n',
  np.float64(1.0)),
 ('Для каких стан является эталоном современная система здравоохранения РФ? Для Зимбабве? Ты тупой? хохлы\n',
  np.float64(1.0)),
 ('В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.\n',


In [121]:
get_10_most_toxic(data.comment, finished_experiments[1])

[('Ну давай разберём всё тобой написанное. Бляядь, вы действительно думаете вы лучше пидорашек? Ну в целом, всё что живёт в рашке - затронуто говномидасом, но никто тут это не признает. сейчас воспитывают массу хороших кодеров В соседнем треде обоссали уже. Иди обтекай. Вы унижаете русских детей в школе Я учился в рашке и у нас был класс, который состоял онли из русачков. Думаешь, что то изменилось? Чурки тебе говна в жопу залили и заставили русачков в классе кошмарить омежных русачков? Я не люблю выражаться фразами нациков, но вы воистину столетия просто сидели в горах и ебали баранов, вас даже народ-пидор смог захватить. Плоховато ты знаешь историю. Когда русня пришла на Кавказ, тут всё уже было поделено османами и персами. А потом РИ наебала персов и постоянно нападала на османов высасывая причины из пальца, в принципе, русачки, что от них ещё было ожидать. В прошлом вы (чеченцы, даги и прочие сорта) были просто дикарями...А если говорить о среднеазиатах, которые бугуртят с оккупаци

In [122]:
get_10_most_toxic(data.comment, finished_experiments[2])

[('Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n',
  np.float64(1.0)),
 ('тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n',
  np.float64(1.0)),
 ('УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РАЗВОРУЮТ КАК ВСЕГДА! УЖЕ ТРЕЩИНАМИ ПОШ Л! ТУПЫЕ КИТАЗЫ НЕ МОГУТ НИЧЕГО НОРМАЛЬНО СДЕЛАТЬ!\n',
  np.float64(1.0)),
 ('Ебать тебя разносит, шизик.\n', np.float64(1.0)),
 ('Уроды!! у нас в семье 3 поколения там родились\n', np.float64(1.0)),
 ('Можем на тебя ещё и модера за безмозглых позвать.\n', np.float64(1.0)),
 ('Всем, кстати, наплевать. Главное - уровень жизни. Вдвойне наплевать на кукареканье тупорылого хохла с форума-помойки.\n',
  np.float64(1.0)),
 ('В обеих странах есть партия войны . И в обеих странах эта война сейчас разжигается и поддерживается Смотрите, парни. И вот после этого руснявого пиздежа пидараны требуют к ним хорошего отношения?

In [14]:
same_normal_results      = []
same_toxic_results       = []
different_normal_results = []
different_toxic_results  = []

# for comment, is_toxic, nb_bow_pred, logreg_bow_pred, logreg_tfidf_pred \
for comment, results in zip(
        Experiment.test.comment,
        zip(
            experiment.test.toxic,
            nb_bow_preds,
            logreg_bow_preds,
            logreg_tfidf_preds
        )
    ):

    elem_to_append = comment.strip() + ": " + " ".join(map(str, results))
    if sum(results) in (0, 4):
        if results[0] == 1:
            same_toxic_results.append(elem_to_append)
        else:
            same_normal_results.append(elem_to_append)
    else:
        if results[0] == 1:
            different_toxic_results.append(elem_to_append)
        else:
            different_normal_results.append(elem_to_append)

In [15]:
len(same_toxic_results), len(same_normal_results), len(different_toxic_results), len(different_normal_results)

(333, 885, 150, 74)

In [16]:
for c in same_toxic_results[:10]:
    print(c)

Взгляните на пизду. Как психически здоровый человек захочет вставлять в эту мерзость? Женская издания это самый страшный орган человека.: 1.0 1.0 1.0 1.0
ты такое же говно как и бомжи, т.к. у них гнилые органы, а у тебя мозг отупел и сгнил.: 1.0 1.0 1.0 1.0
Отправить ты можешь только хуй в свою жопу, лахтодырка.: 1.0 1.0 1.0 1.0
А из хохляндии сколько эмигрировало? А почему ты не говоришь, сколько приехало в Россию? только официальные - а есть ещё те, кто лесами границу переходил, что ли?: 1.0 1.0 1.0 1.0
Ну это типично. Как он вёл? -нормально же: 1.0 1.0 1.0 1.0
Надо просто Тарасов-пидорашек из по выгнать, тогда и генотьбы не будет. Нет свиньи - нет проблемы, как говорил создатель хохлов.: 1.0 1.0 1.0 1.0
Пиздец, случайно зашел в ньюс. Тут у вас какая-то вакханалия творится. Одни отыгрывают за хохлов, другие за ватриотов. Пиздец, пойду отсюда нахуй, а то еще крыша поедет.: 1.0 1.0 1.0 1.0
Мы казаки, ты а пидораха.: 1.0 1.0 1.0 1.0
8.Комментаторы комментируют пост с комментариями комме

In [17]:
for c in same_normal_results[:10]:
    print(c)

Я когда-то давно на рыбном заводе работал. Так там все приходили на работу через проходную. А после выходили через дыру в заборе с полными катомками рыбопродукции. Название у этой дыры было: выходная... Я к чему, если бутылка водки не пролезет через проходную, путь вам в выходную.: 0.0 0.0 0.0 0.0
Глупость говорите, новая УК не отвечает за старую УК. Тем более замена радиатора это вообще не обязанность УК. Граница ответственность УК запирающее устройство, все после него - имущество собственника. Человек захотел себе радиатор поставить, спилив чугунную. За отдельную плату ее поставили, подписали акты, с этого момента радиатор собственника и он отвечает за его содержание и все последствия. Другое дело, что женщина может взыскать убытки с подрядчика, который установил ей кривой радиатор, но сначала ей придется эти убытки понести (то есть выплатить ущерб), да ещё и доказать, что радиатор реально кривой.: 0.0 0.0 0.0 0.0
Ну тут, имхо, от магазина конкретного зависит все.: 0.0 0.0 0.0 0.0
И 

In [18]:
for c in different_toxic_results[:10]:
    print(c)

Блин, , до сих пор хрюкаю и ржу! Спасибо тебе! Ты мне создал хорошее настроение своим уточнением на длительное время! :-D: 1.0 0.0 0.0 0.0
Форчан, Двач, Луркморье. Только вы там поосторожней, у них своя атмосфера.: 1.0 1.0 1.0 0.0
А давайте не путать феминизм и идиотские высказывания тупых и страшных женщин, а? Феминизм-это желание иметь равные права в работе и в жизни, но быть женщиной, слабым прекрасным полом, как и происходит сейчас в нормальном обществе. Радфемки-это чудаковатые бабы, которые просто орут, что мужчины им не нужны и вообще все козлы. Во-вторых, каждая женщина и каждый мужчина имеют право выбирать спутника исходя из своих критериев. Другой вопрос, что под их критерии может рикто не попасть, или идеал их сам не захочет. Но осуждать их за это, смысл?: 1.0 1.0 1.0 0.0
А у меня кандидатка завалилась на электрохромизме, не могут эти твари выдержать без разрушения больше 500 электрохимических циклов без разрушения тонких плёнок( Хотя чему я удивляюсь работы по электрохромиз

In [19]:
for c in different_normal_results[:10]:
    print(c)

А как насчет клеветы? Обвиняют, что виновен в дтп, когда человек не виновен.: 0.0 0.0 1.0 0.0
Что за ролик? Тоже хочу такой звук: 0.0 1.0 0.0 0.0
Хуй пойми за что тебя минусуют. Я ищу вот такую девушку - Да как ты посмел нам указывать, ко-ко-ко, это мои интересы . Что-то странная хуета: высказываешь свою позицию, оказывается ты угнетаешь девушек и вообще должен заткнуться и засунуть своё мнение в задницу.: 0.0 1.0 1.0 1.0
Так же как бандосов. Есть авторитетный человек которому доверяют оба уголовника, тот их и судит.: 0.0 1.0 1.0 1.0
Не все муслимы террористы, но все террористы - муслимы (до недавнего времени): 0.0 1.0 1.0 1.0
Красивый мужик это пидр..: 0.0 1.0 1.0 1.0
Что-то появившееся на небе пугает туристов Кто рассказал Черноброву про это, дух горы, мертвецы дятловцы?: 0.0 1.0 1.0 1.0
меня тут на хуй послали и модератор даже не отреагировал, пришлось банить идиота. А ВЫ говорите, что работает .: 0.0 1.0 0.0 0.0
А вот говорить что сниматься в порно хорошо, нормально, а распространя

#### Комментарий

Токсичность комментариев -- очень субъективная тема, особенно в рамках бинарной классификации. Кого-то может и вот такое задеть: ```"хочешь грудь модератора?"``` <s>(последний пример из списка выше)</s> <i>перезапустил все ячейки и результаты поменялись, но раньше этот пример там был. Надо приучить себя определять рандом сид.</i> Конечно, для такой задачи нужно бы иметь побольше классов.

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [96]:
experiment_params = (

    {
        "classifier_model": MultinomialNB,
        "classifier_args" : {},
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {   
        "classifier_model": LogisticRegression,
        "classifier_args" : {"solver":"liblinear", "penalty":"l2"},
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {
        "classifier_model": DecisionTreeClassifier,
        "classifier_args" : {"min_samples_split":2, "max_depth":26, "class_weight": 'balanced'},
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {
        "classifier_model": RandomForestClassifier,
        "classifier_args" : {"min_samples_split":2, "max_depth":26, "class_weight": 'balanced'},
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

)

In [97]:
finished_experiments = []

for d in experiment_params:
    experiment = Experiment(**d)
    print(f'{d["classifier_model"].__name__}\t{d["vectorizer_model"].__name__}')
    print("F1: ", experiment.get_f1())
    print(experiment.get_report())
    finished_experiments.append(experiment)

MultinomialNB	CountVectorizer
F1:  0.8726114649681529
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94       959
         1.0       0.90      0.85      0.87       483

    accuracy                           0.92      1442
   macro avg       0.91      0.90      0.91      1442
weighted avg       0.92      0.92      0.92      1442



/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LogisticRegression	CountVectorizer
F1:  0.837108953613808
              precision    recall  f1-score   support

         0.0       0.90      0.94      0.92       959
         1.0       0.87      0.80      0.84       483

    accuracy                           0.90      1442
   macro avg       0.89      0.87      0.88      1442
weighted avg       0.89      0.90      0.89      1442

DecisionTreeClassifier	CountVectorizer
F1:  0.6359918200408998
              precision    recall  f1-score   support

         0.0       0.82      0.81      0.81       959
         1.0       0.63      0.64      0.64       483

    accuracy                           0.75      1442
   macro avg       0.72      0.73      0.72      1442
weighted avg       0.75      0.75      0.75      1442

RandomForestClassifier	CountVectorizer
F1:  0.712166172106825
              precision    recall  f1-score   support

         0.0       0.87      0.82      0.84       959
         1.0       0.68      0.75      0.71       483


#### Комментарий

<q>по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75</q>

Кажется, такой возможности нет :(

#### Для дерева решений:

In [51]:
from collections import Counter
import numpy as np

In [45]:
counter = Counter(
    {
        token: importance
        for token, importance
        in zip(
            finished_experiments[2].vectorizer.vocabulary_,
            finished_experiments[2].classifier.feature_importances_
        )
    }
)

In [46]:
counter.most_common(5)

[('(не ', np.float64(0.09549377153881515)),
 ('ят-ко', np.float64(0.07708447535110827)),
 ('редка', np.float64(0.06147589391657986)),
 ('недур', np.float64(0.033763387534206615)),
 ('ато.', np.float64(0.030857429719008116))]

#### Для леса

In [98]:
counter = Counter(
    {
        token: importance
        for token, importance
        in zip(
            finished_experiments[3].vectorizer.vocabulary_,
            finished_experiments[3].classifier.feature_importances_
        )
    }
)

In [99]:
counter.most_common(5)

[('орниш', np.float64(0.004847584581782989)),
 ('ема,', np.float64(0.004449736649807151)),
 (' (не', np.float64(0.004388843863988988)),
 ('ощал', np.float64(0.004067373274740722)),
 ('сс. ', np.float64(0.0037168778427236664))]

#### Для наивного Байеса

In [94]:
counter = Counter(
    {
        token: importance
        for token, importance
        in zip(
            finished_experiments[0].vectorizer.vocabulary_,
            finished_experiments[0].classifier.feature_log_prob_[1]
        )
    }
)

In [95]:
counter.most_common(5)

[('коти', np.float64(-6.2147600948356345)),
 ('амые', np.float64(-6.2612959151249985)),
 ('actic', np.float64(-6.300294121741232)),
 ('щёбу ', np.float64(-6.3978120875995135)),
 ('лстр', np.float64(-6.4046107540585915))]

#### Комментарий

#### Для ЛогРега

In [ ]:
counter = Counter(
    {
        token: importance
        for token, importance
        in zip(
            finished_experiments[1].vectorizer.vocabulary_,
            np.mean(np.abs(finished_experiments[1].classifier.coef_), axis=0)
        )
    }
)

In [63]:
counter.most_common(5)

[(' (не', np.float64(1.0883276543101839)),
 ('ят-ко', np.float64(1.041318533120039)),
 ('авае', np.float64(0.9627045670059449)),
 ('стит,', np.float64(0.9041365699664503)),
 ('репло', np.float64(0.8891025932143486))]